In [38]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D,MaxPool2D, BatchNormalization, Activation

img_width = 320
img_height = 240


input = Input((img_width, img_height, 3))
x = Conv2D(96 , 7, strides = 2, padding = 'same')(input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPool2D(3, strides = 2, padding = 'same')(x)


print(x.shape)

mdl = Model(input, x)

mdl.summary()






(None, 80, 60, 96)
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 320, 240, 3)]     0         
                                                                 
 conv2d_472 (Conv2D)         (None, 160, 120, 96)      14208     
                                                                 
 batch_normalization_462 (Ba  (None, 160, 120, 96)     384       
 tchNormalization)                                               
                                                                 
 activation_17 (Activation)  (None, 160, 120, 96)      0         
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 80, 60, 96)       0         
 g2D)                                                            
                                                                 
Total params: 14,592
Trainable params: 1

In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
# Creating Densenet121
def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,36,24]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(input, output)
    return model
    
input_shape = 320, 240, 3
n_classes = 3
model = densenet(input_shape,n_classes)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 320, 240, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_303 (Conv2D)            (None, 160, 120, 64  9472        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_19 (MaxPooling2D  (None, 80, 60, 64)  0           ['conv2d_303[0][0]']             
 )                                                                                          